In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
import xlrd
import seaborn as sns
%matplotlib inline
from sklearn.feature_extraction.text import CountVectorizer
import re, string, unicodedata
import os
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.feature import Word2Vec
import spacy
import math
import nbconvert
%matplotlib inline
import glob
#import gensim
import nltk
import inflection as inflect
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from string import ascii_lowercase
import itertools, snowballstemmer
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.feature import Word2Vec
import spacy
from matplotlib.colors import rgb2hex
from matplotlib.patches import Polygon
from pyspark.sql.session import SparkSession
# instantiate Spark
spark = SparkSession.builder.getOrCreate()

In [ ]:
# Read the input csv data format
drug_reaction_inputdf = pd.read_csv("drug_adverse_report_data.csv",na_values=' ')

In [ ]:
# check the number of rows and columns and account for total input data imported

drug_reaction_inputdf.shape

In [ ]:
# Quality control for data type and the number of non null values in each column

drug_reaction_inputdf.info()

In [ ]:
drug_reaction_inputdf.isnull().sum(axis = 0)/len(drug_reaction_inputdf)*100

In [ ]:
# count of zero values

row_zero_count = (df[df == 0].count(axis=1)/len(df.columns))
raw dataframe after dropping redaundant columns
column_zero_count = (drug_reaction_inputdf[drug_reaction_inputdf == 0].count(axis=0)/len(drug_reaction_inputdf.index))*100
column_zero_count

In [ ]:
drug_reaction_inputdf.nunique()

In [ ]:
# show column 0 to 15 and 3 rows of dataframe
drug_reaction_inputdf.iloc[:,0:15].head(3)

In [ ]:
# show column 16 to 30 and 3 rows of dataframe

drug_reaction_inputdf.iloc[:,16:30].head(3)

In [ ]:
# show column 31 to 45 and 3 rows of dataframe

drug_reaction_inputdf.iloc[:,31:45].head(3)

In [ ]:
# show column 36 to 54 and 3 rows of dataframe

drug_reaction_inputdf.iloc[:,36:54].head(3)

In [ ]:
# select numerical variables to display summary statistics(df.loc[:, df.dtypes == np.float64])

selected_int_df = ['Patientsex','Patientonsetage','Drugenddate','Drugtreatmentduration','Drugcharacterization','Serious']

drug_reaction_intgerdf = drug_reaction_inputdf[selected_int_df]

In [ ]:
drug_reaction_intgerdf.describe()

In [ ]:
# count of zero values

row_zero_count = (df[df == 0].count(axis=1)/len(df.columns))
raw dataframe after dropping redaundant columns
column_zero_count = (drug_reaction_inputdf[drug_reaction_inputdf == 0].count(axis=0)/len(drug_reaction_inputdf.index))*100
column_zero_count

In [ ]:
drug_reaction_inputdf.nunique()

In [ ]:
#sumary of data

drug_reaction_inputdf.iloc[:,36:46].describe()

In [ ]:
# Data cleaning
# The data cleaning process is implemented to drop redundant and duplicated variables that are not required,
# also create uniform datatypes in each column of dataframe.
# This process of cleaning each column will remove meta characters, numerical value
# in text columns and texts from numeric columns. This will produce same data type for all values of a
# variable. It will increase accuracy of plots, data engineering and modelling.
# The unique values in each columns will be examined for cleaning columns where necesary.
# functions are creted to clean numeric and objects data types respectively
# The count of unique values will be displayed before and after cleaning to check any deviation

In [ ]:
# select and remove duplicate columns

drug_reaction_df1 = drug_reaction_inputdf.copy()

duplicate_cols = ['drug Index (generated)','generic name Index (generated)','pharm class cs Index (generated)','pharm class epc Index (generated)',
'pharm class pe Index (generated)', 'reaction Index (generated)','results Index (generated)',
 'substance name Index (generated)','route Index (generated)']

drug_reaction_df1 = drug_reaction_df1.drop(drug_reaction_df1[duplicate_cols], axis = 1)

In [ ]:
# check Null values in each column, the columns with more than 70% of null values will be removed
# check the percentage of null values
#1.4 Function to calculate percentage of NaN missing values in each variable of the dataset. columns where 70 to 100% of values are Null are dropped at this stage

def nan_percentage(df):
    ''' calcute the percentage of NaN values for each column in the dataframe and return only columns containing NaN and the percentage'''
    
    nandf = df.loc[:, df.isnull().any()]  # get columns containing nan values
    nan_percent = nandf.isnull().sum(axis = 0)/len(df)*100  # percentage of nan values
    
    return nan_percent
# call the function


In [ ]:
# apply the nan_percent function

df = drug_reaction_df1

nan_percentage()

In [ ]:
null_columns = ['Drugauthorizationnumb','Drugbatchnumb','Companynumb','Drugtreatmentduration',
                'Drugtreatmentdurationunit','Companynumb','Seriousnessdisabling','Seriousnesslifethreatening',
                'Seriousnesscongenitalanomali','Reportercountry','Safetyreportid']

drug_reaction_df2 = drug_reaction_df1.copy()

drug_reaction_df2 = drug_reaction_df2.drop(drug_reaction_df2[null_columns], axis = 1)

In [ ]:
# apply the nan_percent function

df = drug_reaction_df2

nan_percentage()

In [ ]:
# Map the case lowering function to all column names #map(str.lower, df.columns) 

drug_reaction_df3 = drug_reaction_df2.copy()

# lowercase column name amd remove space

drug_reaction_df3.columns = drug_reaction_df3.columns.str.lower().str.replace(' ', '')

drug_reaction_df3.columns

In [ ]:
# 2.1 create a function to clean integer type data

def integer_clean(df):
    '''return dataframe for integer data to retain only the digits (and dot and minus sign).
    This would remove characters, alphabets or anything that is not defined in to_replace attribute
    quality control data after cleaning
    '''
    df = df.loc[:, df.dtypes == np.int64].replace(regex=True, to_replace=r'[^0-9.\-]', value=r'')
    
    return df

# call function

In [ ]:
# 2.2 create a function to clean float type data.

def float_clean(df):
    '''return dataframe for integer data to retain only the digits (and dot and minus sign).
    This would remove characters, alphabets or anything that is not defined in to_replace attribute
    quality control data after cleaning
    '''
    df = df.loc[:, df.dtypes == np.float64].replace(regex=True, to_replace=r'[^0-9.\-]', value=r'')
    
    return df
   
#call function

In [ ]:
# 2.3 create a function to clean string data.

def clean_text(df = df, col= 'make'):
    '''This function returns a pandas list object of the given dataframe and column.
    The values of the column will be changed to lower case, remove spaces, punctuations and numbers,
    replace lines with non ascii characters that might exist'''
    
    df[col]= df[col].str.strip().str.lower().str.replace(' ', '').str.replace('[^\w\s]','').apply(lambda x: x.translate(string.punctuation)).apply(lambda x: x.translate(string.digits)).str.replace('[#%\'()*+,-./:;<=>?@\[\]^_`{|}~1234567890’”“′‘\\\]','').str.replace(r'[^\x00-\x7f]', '')
    
    return df[col]
    
#call function

In [ ]:
# text data columns, check unique values of each text column before transformation. 

drug_reaction_textdf3 = drug_reaction_df3.loc[:, drug_reaction_df3.dtypes == 'object'].copy()

drug_reaction_textdf3.nunique()

In [ ]:
"""
Exoloration Data Analyses
The sub sets and groups of variables were visualised to gain insight on data distribution and
relationship. The questions that guided the plots include.

2.What are the most frequent words in the Drug dosage text?

3.How do the categories vary with target variable?

4.Which age category had more death adverse reaction?

5.Which medicine products and route of intake were more involved in death drug adverse reaction?

6.What is the count ratio of target variable? 

7.What is the effect of drug treatment duration on death drug adverse reaction?

8.Is there a difference based on gender, weight and age on death adverse drug reaction?
"""
a = 'a'

In [ ]:
 # 3.0 Exploratory Data Analysis and visualisation.
'''
The plots are produced using Matplotlib and Seaborn libraries

give a differentiation or separation of the samples in the binary classifier.

The visualisation will show relationship between input features or variables.

Visualise data distribution of each variable for skew correction

This will also help to discover trend and patterns in the data and to understand data characteristics.

The Analysis is also aimed at discovering relationships in data engineering choice.

Plot include univariate plots using Histogram, Barplot, Bivariate plots such as Boxplots, Multivariate scatter plots and cluster plots.
'''
plot ='plot'


In [ ]:
# The target variable for classification is seriousnessdeath
# the value 1 represents death event due to adverse drug reation
# 

drug_reaction_df33 = drug_reaction_df3.copy()

drug_reaction_df33.seriousnessdeath = drug_reaction_df33.seriousnessdeath.fillna(0) # fill non death event with 0

drug_reaction_df33.seriousnessdeath.unique()

In [ ]:
drug_reaction_df33.pharmclassepc.unique()[:10]

In [ ]:
#3.1 BarPlot of target variable 'seriousnessdeath'
plt.figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')
sns.catplot(x="seriousnessdeath", kind="count",palette="ch:.25", data=drug_reaction_df33)
plt.title('Bar plot of Target variable seriousnessdeath')
plt.xlabel('seriousnessdeath')
plt.ylabel('Count of Values')
plt.xticks(rotation='vertical')
plt.legend(loc='best')
plt.show()

In [ ]:
# The data is imbalanced
# The number of drug reations that resulted in death are much less than non death adverse reation events 
"""
pharmclasspe          28
route                 29
"""
a='a'

In [ ]:
np.min(drug_reaction_df33.patientonsetage), np.max(drug_reaction_df33.patientonsetage)

In [ ]:
# count of categorical values

drug_reaction_df33['pharmclasspe_count'].unique()

In [ ]:
# Factor plot 
plt.figure(num=None, dpi=80,figsize =(14,6), facecolor='w', edgecolor='k')
sns.catplot('pharmclasspe', 'patientonsetage', hue='seriousnessdeath', data=drug_reaction_df33, kind='bar')
plt.title('Patient experience subset by seriousness death')
plt.xlabel('pharm class physiological experience')
plt.ylabel('patient age')
plt.xticks(rotation='vertical')
plt.legend(loc='best')
plt.show()

In [ ]:
# Factor plot 
plt.figure(num=None, dpi=80,figsize =(14,6), facecolor='w', edgecolor='k')
sns.catplot('route', 'patientweight', hue='seriousnessdeath', data=drug_reaction_df33, kind='bar')
plt.title('Drug ntake route')
plt.xlabel('route')
plt.ylabel('patientweight')
plt.xticks(rotation='vertical')
plt.legend(loc='best')
plt.show()

In [ ]:
# Histogram of 'patientweight'

# Multiple plots boxplot and histograms in the same window
# Cut the window in 2 parts
f, (ax_box, ax_hist) = plt.subplots(2, figsize =(14,6),sharex=True, gridspec_kw={"height_ratios": (.15, .85)})
 
# Add a graph in each part
sns.boxplot(drug_reaction_df33["patientweight"], ax=ax_box)
sns.distplot(drug_reaction_df33["patientweight"], ax=ax_hist) 
# Remove x axis name for the boxplot
ax_box.set(xlabel='')
plt.title('Boxplot and Histogram of patientweight')
plt.xlabel('patientweight')
plt.ylabel('frequency ditribution')
plt.xticks(rotation='vertical')
plt.legend(loc='best')
plt.show()

In [ ]:
# patientweight distribution is skwed and shows outliers hence has to be transformed
# plot histogram of log transformation of EngineSize
# Add 1 to replace zero before log transform

drug_reaction_df33.loc[:,"patientweightLog"] = (drug_reaction_df33.loc[:,"patientweight"]+1).apply(np.log)

In [ ]:
# Histogram of 'EngineSizeLog'
# Multiple histograms in the same window
# Cut the window in 2 parts
f, (ax_box, ax_hist) = plt.subplots(2, figsize =(14,6),sharex=True, gridspec_kw={"height_ratios": (.15, .85)})
 
# Add a graph in each part
sns.boxplot(drug_reaction_df33["patientweightLog"], ax=ax_box)
sns.distplot(drug_reaction_df33["patientweightLog"], ax=ax_hist) 
# Remove x axis name for the boxplot
ax_box.set(xlabel='')
plt.title('Boxplot and Histogram of patientweightLog')
plt.xlabel('patientweightLog')
plt.ylabel('frequency density')
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
drug_reaction_df33.patientsex.unique()

In [ ]:
# grouped boxplot
drug_reaction_df33.patientsex = drug_reaction_df33.patientsex.fillna(0)
sns.set(style="ticks", palette="pastel")

# Draw a nested boxplot to show bills by day and time
plt.figure(num=None, dpi=80,figsize =(14,6), facecolor='w', edgecolor='k')
f, axes = plt.subplots(1, 1, figsize=(14, 14), sharex=True)
sns.boxplot(x="patientsex", y="patientweight",
            hue="seriousnessdeath", palette=["m", "g"],
            data=drug_reaction_df33)
sns.despine(offset=10, trim=True)
plt.title('Boxplot of patientsex using patientweight, seriousnessdeath as legend' )
plt.xlabel('patientsex')
plt.ylabel('patientweight')
plt.xticks(rotation='vertical')
plt.legend(loc='best')
plt.show()

In [ ]:
# grouped boxplot

sns.set(style="ticks", palette="pastel")

# Draw a nested boxplot to show bills by day and time
plt.figure(num=None, dpi=80,figsize =(14,6), facecolor='w', edgecolor='k')
f, axes = plt.subplots(1, 1, figsize=(14, 14), sharex=True)
sns.boxplot(x="route", y="patientweight",
            hue="seriousnessdeath", palette=["m", "g"],
            data=drug_reaction_df33)
sns.despine(offset=10, trim=True)
plt.title('Boxplot of route using patientweight, seriousnessdeath as legend' )
plt.xlabel('route')
plt.ylabel('patientweight')
plt.xticks(rotation='vertical')
plt.legend(loc='best')
plt.show()

In [ ]:
#Feature extraction, transformation and engineering
# feature were extracted using word3vec in text variables with very high number of categorical values
# Also numerical values such as Age was also categorised by biining into groups as seriousness of adverse drug effect
# varies with age groups.

In [ ]:
drug_reaction_df3.info()

In [ ]:
#  check the unique values of Drugdosagetext

drug_reaction_df3.drugdosagetext[10:1000].unique()

In [ ]:
# comment
# There are many unique values in the text columns,the data will be preprocessed by 
# replacing missing value
# lower casing of the text
# vectorisation of the values using word2vec 

In [ ]:
# change values to lower case and replace missing values

drug_reaction_df3a = drug_reaction_df3.copy()

drug_reaction_df3a.drugdosagetext = drug_reaction_df3a.drugdosagetext.str.lower().fillna('missing') # lower casing

# display for quality control
drug_reaction_df3a.drugdosagetext[10:1000].unique()


In [ ]:
# create a function to implement vectorisation of each text column
# Comment The word2vec will be implemented using sparksession 
# A function is created to vectorise each of the ten text columns

# 2.3 create a function to vectorise each text column.

def text_vectorize():
    '''This function returns a pandas dataframe and columns of word tokens and the vectors.
    implement word2vec using spark session'''
    
    # select the tokenised word
    word_df = tok_df.select('words') 
    
    # Learn a mapping from words to Vectors.
    text_word2Vec = Word2Vec(vectorSize=3, minCount=0, inputCol="words", outputCol="result")
    
    # fit model using data
    text_model = text_word2Vec.fit(word_df)
    
    # transform using fitted model to output vectors
    vec_df = text_model.transform(word_df)
    
    return vec_df

    

In [ ]:
# vectorize drugdosagetext column

dosage_text_df = drug_reaction_df3a.loc[:,['drugdosagetext']].copy()

# create pyspark dataframe
dosage_text_spark = spark.createDataFrame(dosage_text_df)
# tokenize
tokenizer = Tokenizer(inputCol="drugdosagetext", outputCol="words") # tokenise the data in spark_df

tok_df = tokenizer.transform(dosage_text_spark) # tokenize text

# word vectors using function
dosage_vec_df = text_vectorize()

# display for QC of word vectors

for row in dosage_vec_df.take(2):
    print(row)

In [ ]:
# Add the word vectors to dataframe

# convert spark word vector list to pandas 
dosage_pd_df = dosage_vec_df.toPandas()

# select result and create a list 
dosage_vec_list = dosage_pd_df['result'].tolist()   # convert to python list

# convert to pandas dataframe
dosage_vec_pd_df = pd.DataFrame(dosage_vec_list, columns=['dosagevec1', 'dosagevec2', 'dosagevec3'], index = range(92130)) # to datafram

# set input dataframe index range
drug_reaction_df3a.index = range(92130)

# join the word vector columns to the main dataframe
drug_reaction_df4 = pd.concat([drug_reaction_df3a, dosage_vec_pd_df], axis=1)

drug_reaction_df4.head(3)

In [ ]:
# medicinalproduct text column
#change values to lower case and replace missing values

drug_reaction_df5 = drug_reaction_df4.copy()

drug_reaction_df5.medicinalproduct = drug_reaction_df5.medicinalproduct.str.lower().fillna('missing') # lower casing

# display for quality control
drug_reaction_df5.medicinalproduct[0:100].unique()

In [ ]:
# vectorize medicinalproduct

medicine_text_df = drug_reaction_df5.loc[:,['medicinalproduct']].copy()

# create pyspark dataframe
medicine_text_spark = spark.createDataFrame(medicine_text_df)

tokenizer = Tokenizer(inputCol="medicinalproduct", outputCol="words") # tokenise the data in spark_df

tok_df = tokenizer.transform(medicine_text_spark) # tokenize text
# output word vectors using function
medicine_vec_df = text_vectorize()

# display for QC of word vectors

for row in medicine_vec_df.take(2):
    print(row)

In [ ]:
# Add the word vectors to dataframe

# convert spark word vector list to pandas 
medicine_pd_df = medicine_vec_df.toPandas()

# select result and create a list 
medicine_vec_list = medicine_pd_df['result'].tolist()   # convert to python list

# convert to pandas dataframe
medicine_vec_pd_df = pd.DataFrame(medicine_vec_list, columns=['medvec1', 'medvec2', 'medvec3'], index = range(92130)) # to datafram

# set input dataframe index range
drug_reaction_df5.index = range(92130)

# join the word vector columns to the main dataframe
drug_reaction_df6 = pd.concat([drug_reaction_df5, medicine_vec_pd_df], axis=1)

drug_reaction_df6.head(3)


In [ ]:
# vectorise reactionmeddrapt

# change values to lower case and replace missing values

drug_reaction_df7 = drug_reaction_df6.copy()

drug_reaction_df7.reactionmeddrapt = drug_reaction_df7.reactionmeddrapt.str.lower().fillna('missing') # lower casing

# display for quality control
drug_reaction_df7.reactionmeddrapt[10:100].unique()

In [ ]:
# vectorize medicinalproduct

reactionmed_text_df = drug_reaction_df7.loc[:,['reactionmeddrapt']].copy()

# create pyspark dataframe
reactionmed_text_spark = spark.createDataFrame(reactionmed_text_df)

tokenizer = Tokenizer(inputCol="reactionmeddrapt", outputCol="words") # tokenise the data in spark_df

tok_df = tokenizer.transform(reactionmed_text_spark) # tokenize text
# output word vectors using function
reactionmed_vec_df = text_vectorize()

# display for QC of word vectors

for row in reactionmed_vec_df.take(2):
    print(row)


In [ ]:
# vectorize reactionmeddrapt

# Add the word vectors to dataframe

# convert spark word vector list to pandas 
reaction_pd_df = reactionmed_vec_df.toPandas()

# select result and create a list 
reaction_vec_list = reaction_pd_df['result'].tolist()   # convert to python list

# convert to pandas dataframe
reaction_vec_pd_df = pd.DataFrame(reaction_vec_list, columns=['reactionvec1', 'reactionvec2', 'reactionvec3'], index = range(92130)) # to datafram

# set input dataframe index range
drug_reaction_df7.index = range(92130)

# join the word vector columns to the main dataframe
drug_reaction_df8 = pd.concat([drug_reaction_df7, reaction_vec_pd_df], axis=1)

drug_reaction_df8.head(3)

In [ ]:
drug_reaction_df8.info()

In [ ]:
# vectorise drugindication

# change values to lower case and replace missing values

drug_reaction_df9 = drug_reaction_df8.copy()

drug_reaction_df9.drugindication = drug_reaction_df9.drugindication.str.lower().fillna('missing') # lower casing

# display for quality control
drug_reaction_df9.drugindication[10:100].unique()

In [ ]:
# vectorize drugindication

indication_text_df = drug_reaction_df9.loc[:,['drugindication']].copy()

# create pyspark dataframe
indication_spark = spark.createDataFrame(indication_text_df)

tokenizer = Tokenizer(inputCol="drugindication", outputCol="words") # tokenise the data in spark_df

tok_df = tokenizer.transform(indication_spark) # tokenize text
# output word vectors using function
indication_vec_df = text_vectorize()

# display for QC of word vectors

for row in indication_vec_df.take(2):
    print(row)

In [ ]:
# Add the word vectors to dataframe

# convert spark word vector list to pandas 
indication_pd_df = indication_vec_df.toPandas()

# select result and create a list 
indication_vec_list = indication_pd_df['result'].tolist()   # convert to python list

# convert to pandas dataframe
indication_vec_pd_df = pd.DataFrame(indication_vec_list, columns=['indicationvec1', 'indicationvec2', 'indicationvec3'], index = range(92130)) # to datafram

# set input dataframe index range
drug_reaction_df9.index = range(92130)

# join the word vector columns to the main dataframe
drug_reaction_df10 = pd.concat([drug_reaction_df9, reaction_vec_pd_df], axis=1)

drug_reaction_df10.head(3)

In [ ]:
# vectorise substancename

# change values to lower case and replace missing values

drug_reaction_df11 = drug_reaction_df10.copy()

drug_reaction_df11.substancename = drug_reaction_df11.substancename.str.lower().fillna('missing') # lower casing

# display for quality control
drug_reaction_df11.substancename[10:100].unique()

In [ ]:
# vectorize substancename

substance_text_df = drug_reaction_df11.loc[:,['substancename']].copy()

# create pyspark dataframe
substance_spark = spark.createDataFrame(substance_text_df)

tokenizer = Tokenizer(inputCol="substancename", outputCol="words") # tokenise the data in spark_df

tok_df = tokenizer.transform(substance_spark) # tokenize text
# output word vectors using function
substance_vec_df = text_vectorize()

# display for QC of word vectors

for row in substance_vec_df.take(2):
    print(row)

In [ ]:
# Add the word vectors to dataframe

# convert spark word vector list to pandas 
substance_pd_df = substance_vec_df.toPandas()

# select result and create a list 
substance_vec_list = substance_pd_df['result'].tolist()   # convert to python list

# convert to pandas dataframe
substance_vec_pd_df = pd.DataFrame(substance_vec_list, columns=['substancevec1', 'substancevec2', 'substancevec3'], index = range(92130)) # to datafram

# set input dataframe index range
drug_reaction_df11.index = range(92130)

# join the word vector columns to the main dataframe
drug_reaction_df12 = pd.concat([drug_reaction_df11, substance_vec_pd_df], axis=1)

drug_reaction_df12.head(3)

In [ ]:
# change values to lower case and replace missing values in pharmclassepc

drug_reaction_df13 = drug_reaction_df12.copy()

drug_reaction_df13.pharmclassepc = drug_reaction_df13.pharmclassepc.str.lower().fillna('missing') # lower casing

# display for quality control
drug_reaction_df13.pharmclassepc[10:100].unique()

In [ ]:
# vectorize substancename

classepc_text_df = drug_reaction_df13.loc[:,['pharmclassepc']].copy()

# create pyspark dataframe
classepc_spark = spark.createDataFrame(classepc_text_df)

tokenizer = Tokenizer(inputCol="pharmclassepc", outputCol="words") # tokenise the data in spark_df

tok_df = tokenizer.transform(classepc_spark) # tokenize text
# output word vectors using function
classepc_vec_df = text_vectorize()

# display for QC of word vectors

for row in classepc_vec_df.take(2):
    print(row)

In [ ]:
# Add the word vectors to dataframe

# convert spark word vector list to pandas 
classepc_pd_df = classepc_vec_df.toPandas()

# select result and create a list 
classepc_vec_list = classepc_pd_df['result'].tolist()   # convert to python list

# convert to pandas dataframe
classepc_vec_pd_df = pd.DataFrame(classepc_vec_list, columns=['classepcvec1', 'classepcvec2', 'classepcvec3'], index = range(92130)) # to datafram

# set input dataframe index range
drug_reaction_df13.index = range(92130)

# join the word vector columns to the main dataframe
drug_reaction_df14 = pd.concat([drug_reaction_df13, classepc_vec_pd_df], axis=1)

drug_reaction_df14.head(3)

In [ ]:
# vectorise pharmclasscs column

# change values to lower case and replace missing values in pharmclassepc

drug_reaction_df15 = drug_reaction_df14.copy()

drug_reaction_df15.pharmclasscs = drug_reaction_df15.pharmclasscs.str.lower().fillna('missing') # lower casing

# display for quality control
drug_reaction_df15.pharmclasscs[10:100].unique()

In [ ]:
# vectorize pharmclasscs

pharmclasscs_text_df = drug_reaction_df15.loc[:,['pharmclasscs']].copy()

# create pyspark dataframe
pharmclasscs_spark = spark.createDataFrame(pharmclasscs_text_df)

tokenizer = Tokenizer(inputCol="pharmclasscs", outputCol="words") # tokenise the data in spark_df

tok_df = tokenizer.transform(pharmclasscs_spark) # tokenize text

# output word vectors using function
pharmclasscs_vec_df = text_vectorize()

# display for QC of word vectors

for row in pharmclasscs_vec_df.take(2):
    print(row)

In [ ]:
# Add the word vectors to dataframe

# convert spark word vector list to pandas 
pharmclasscs_pd_df = pharmclasscs_vec_df.toPandas()

# select result and create a list 
pharmclasscs_vec_list = pharmclasscs_pd_df['result'].tolist()   # convert to python list

# convert to pandas dataframe
pharmclasscs_vec_pd_df = pd.DataFrame(pharmclasscs_vec_list, columns=['pharmclasscsvec1', 'pharmclasscsvec2', 'pharmclasscsvec3'], index = range(92130)) # to datafram

# set input dataframe index range
drug_reaction_df15.index = range(92130)

# join the word vector columns to the main dataframe
drug_reaction_df16 = pd.concat([drug_reaction_df15, pharmclasscs_vec_pd_df], axis=1)

drug_reaction_df16.head(3)

In [ ]:
# change values to lower case and replace missing values in pharmclasspe

drug_reaction_df17 = drug_reaction_df16.copy()

drug_reaction_df17.pharmclasspe = drug_reaction_df17.pharmclasspe.str.lower().fillna('missing') # lower casing

# display for quality control
drug_reaction_df17.pharmclasspe[10:100].unique()

In [ ]:
# vectorize  pharmclasspe

pharmclasspe_text_df = drug_reaction_df17.loc[:,['pharmclasspe']].copy()

# create pyspark dataframe
pharmclasspe_spark = spark.createDataFrame( pharmclasspe_text_df)

tokenizer = Tokenizer(inputCol="pharmclasspe", outputCol="words") # tokenise the data in spark_df

tok_df = tokenizer.transform(pharmclasspe_spark) # tokenize text

# output word vectors using function
pharmclasspe_vec_df = text_vectorize()

# display for QC of word vectors

for row in pharmclasspe_vec_df.take(2):
    print(row)

In [ ]:
# Add the word vectors to dataframe

# convert spark word vector list to pandas 
pharmclasspe_pd_df = pharmclasspe_vec_df.toPandas()

# select result and create a list 
pharmclasspe_vec_list = pharmclasspe_pd_df['result'].tolist()   # convert to python list

# convert to pandas dataframe
pharmclasspe_vec_pd_df = pd.DataFrame(pharmclasspe_vec_list, columns=['pharmclasspevec1', 'pharmclasspevec2', 'pharmclasspevec3'], index = range(92130)) # to datafram

# set input dataframe index range
drug_reaction_df17.index = range(92130)

# join the word vector columns to the main dataframe
drug_reaction_df18 = pd.concat([drug_reaction_df17, pharmclasspe_vec_pd_df], axis=1)

drug_reaction_df18.head(3)

In [ ]:
# change values to lower case and replace missing values in route

drug_reaction_df19 = drug_reaction_df18.copy()

drug_reaction_df19.route = drug_reaction_df19.route.str.lower().fillna('missing') # lower casing

# display for quality control
drug_reaction_df19.route.unique()

In [ ]:
drug_reaction_df19.info()

In [ ]:
# vectorize  route

route_text_df = drug_reaction_df19.loc[:,['route']].copy()

# create pyspark dataframe
route_spark = spark.createDataFrame(route_text_df)

tokenizer = Tokenizer(inputCol="route", outputCol="words") # tokenise the data in spark_df

tok_df = tokenizer.transform(route_spark) # tokenize text

# output word vectors using function
route_vec_df = text_vectorize()

# display for QC of word vectors

for row in route_vec_df.take(2):
    print(row)

In [ ]:
# Add the word vectors to dataframe

# convert spark word vector list to pandas 
route_pd_df = route_vec_df.toPandas()

# select result and create a list 
route_vec_list = route_pd_df['result'].tolist()   # convert to python list

# convert to pandas dataframe
route_vec_pd_df = pd.DataFrame(route_vec_list, columns=['routevec1', 'routevec2', 'routevec3'], index = range(92130)) # to datafram

# set input dataframe index range
drug_reaction_df19.index = range(92130)

# join the word vector columns to the main dataframe
drug_reaction_df20 = pd.concat([drug_reaction_df19, route_vec_pd_df], axis=1)

drug_reaction_df20.head(3)

In [ ]:
#save out data to csv

# Save the cleaned dataframe to disk1 as a csv file

drug_reaction_df20.to_csv("drug_reaction_df20.csv")   # save out the data to disc


In [ ]:
# drop vectorized columns
# drop vectorised columns
text_drop_cols = ['drugdosagetext','drugindication','genericname','medicinalproduct','pharmclasscs','pharmclasscs',
                  'pharmclassepc','pharmclasspe','reactionmeddrapt','route','substancename']
drug_reaction_df21 = drug_reaction_df20.copy()
drug_reaction_df21 = drug_reaction_df21.drop(drug_reaction_df21[text_drop_cols], axis = 1)

In [ ]:
drug_reaction_df21.info()

In [ ]:
# patientonsetage and patientweight contain missing values
# Hence replace missing values with median and bin the variables
# The variables containing missing values are not skews hence the column mean value will be used to imput missing values

drug_reaction_df22 = drug_reaction_df21.copy()

drug_reaction_df22 = drug_reaction_df22.iloc[:,1:54]

# missing value for patientweight
drug_reaction_df22['patientweight'] = drug_reaction_df22['patientweight'].fillna((drug_reaction_df22['patientweight'].median()))

# missing value for patientonsetage
drug_reaction_df22['patientonsetage'] = drug_reaction_df22['patientonsetage'].fillna((drug_reaction_df22['patientonsetage'].median()))

# fill with mean as the values are not skewed

drug_reaction_df22['patientsex'] = drug_reaction_df22['patientsex'].fillna((drug_reaction_df22['patientsex'].mean()))

drug_reaction_df22['qualification'] = drug_reaction_df22['qualification'].fillna((drug_reaction_df22['qualification'].mean()))

drug_reaction_df22['serious'] = drug_reaction_df22['serious'].fillna((drug_reaction_df22['serious'].mean()))

# seriousnessdeath is the target variable,values are 1 and 0
drug_reaction_df22.seriousnessdeath = drug_reaction_df22.seriousnessdeath.fillna(0)

In [ ]:
drug_reaction_df22.info()

In [ ]:
#
count_df = drug_reaction_df22[['patientonsetage','patientweight']].copy()
count_df.describe()

In [ ]:
# bining age and weight
# bining for patientonsetage

drug_reaction_df22['patientonsetagebin'] = pd.cut(drug_reaction_df22.patientonsetage,bins=4,labels=range(1, 5), retbins=False,include_lowest=True)

drug_reaction_df22['patientonsetagebin'] = drug_reaction_df22['patientonsetagebin'].astype('int64')

drug_reaction_df22['patientweightbin'] = pd.cut(drug_reaction_df22.patientweight,bins=4,labels=range(1, 5), retbins=False,include_lowest=True)

drug_reaction_df22['patientweightbin'] = drug_reaction_df22['patientweightbin'].astype('int64')

In [ ]:
drug_reaction_df22['patientonsetagebin'].unique()

In [ ]:
drug_reaction_df22.info()

In [ ]:
# drop columns after feature extraction

# drop patientage and patientweight
drug_reaction_df23 = drug_reaction_df22.copy()
drug_reaction_df23 = drug_reaction_df23.drop(drug_reaction_df23[['patientonsetage','patientweight','drugadministrationroute',
                                                                 'drugenddate','drugenddateformat','drugstartdate',
                                                                 'drugstartdateformat','patientonsetageunit','receiptdate',
                                                                 'receiptdateformat','receivedateformat',
                                                                 'seriousnessother','seriousnesshospitalization',
                                                                 'seriousnessother','transmissiondate',
                                                                 'transmissiondateformat','receivedate','seriousnessdeath']], axis = 1)
drug_reaction_df23['seriousnessdeath'] = drug_reaction_df22['seriousnessdeath']

In [ ]:
# plot of correlation matrix

# Compute the correlation matrix
corr = drug_reaction_df23.corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

In [ ]:
# Features and target data, split the model training and test data

drug_reaction_df24 = drug_reaction_df23.copy()
features_df= drug_reaction_df24.drop(['seriousnessdeath'], axis = 1) # features output

target_df = drug_reaction_df24['seriousnessdeath']  # target output


In [ ]:
features_df.info()

In [ ]:
drug_reaction_df24['seriousnessdeath'].value_counts()

In [ ]:
target_df.shape

In [ ]:
# features scaling
# Feature scaling improves the convergence of steepest descent algorithms, which do not possess 
#the property of scale invariance
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

std_scale = preprocessing.StandardScaler().fit(features_df)
df_std = std_scale.transform(features_df)
features_scaled_df = df_std


In [ ]:
#5.1  Train test splitting
from __future__ import print_function
from sklearn.model_selection import train_test_split
from sklearn import metrics

# Make a train/test split using 20% test size
X_train, X_test, y_train, y_test = train_test_split(features_scaled_df, target_df,
                                                    test_size=0.20,
                                                    random_state=21)

print("X_train dataset: ", X_train.shape)
print("y_train dataset: ", y_train.shape)
print("X_test dataset: ", X_test.shape)
print("y_test dataset: ", y_test.shape)

In [ ]:
# Resample: Oversampling the training data using SMOTE
from imblearn.over_sampling import SMOTE

print("Before OverSampling, counts of label '1': {}".format(sum(y_train==1)))
print("Before OverSampling, counts of label '0': {} \n".format(sum(y_train==0)))

sm = SMOTE(random_state=2)
X_train_res, y_train_res = sm.fit_sample(X_train, y_train.ravel())

print('After OverSampling, the shape of train_X: {}'.format(X_train_res.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(y_train_res.shape))

print("After OverSampling, counts of label '1': {}".format(sum(y_train_res==1)))
print("After OverSampling, counts of label '0': {}".format(sum(y_train_res==0)))

In [ ]:
# Feature Seletion  (Recursive Feature Elimination)
# Recursive Feature Elimination (RFE) repeatedly constructs a model and choose eitherthe best or worst 
# performing features. The goal of RFE is to select features by recursively considering smaller 
# and smaller sets of features.

y=target_df
X=features_df
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(solver='lbfgs',max_iter=200, random_state=0)
rfe = RFE(logreg, 20)
rfe = rfe.fit(X_train, y_train)
print(rfe.support_)
print(rfe.ranking_)

In [ ]:
rfe_score = rfe.score(X_test,y_test)
print(rfe_score)
print(sorted(zip(map(lambda x: round(x, 4),rfe_score), 
                 names), reverse=True))

In [ ]:
# Create logistic regression model using the resample data
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, precision_recall_curve, auc, roc_auc_score, roc_curve, recall_score,classification_report


In [ ]:
# 5.0 Model fitting
#In the binary classification task, the following models are fitted and comapred using different 
# evaluation metrics. Logistic regression, NaiveBias, SVm, RandomForest, XGboost
# Gradient Descent parameter optimisation

In [ ]:
# Logistic regaression
# Allfeatures are selected as good for modelling. 

lr = LogisticRegression(C=1.0,solver='lbfgs',max_iter=250, random_state=0)
lr.fit(X_train, y_train)

#Score is Mean Accuracy
logistic_score = lr.score(X_test,y_test)
print( 'logisticregression score: ', logistic_score)

In [ ]:
# Grid search optimisation
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.model_selection import RandomizedSearchCV

# Create logistic regression
lrg = LogisticRegression(random_state=0,max_iter=200,solver='lbfgs' )             # initialize the model

# parameter grid
max_iter=[200,250,300,350]
C = [1.0,1.5,2.0,2.5]
# Create hyperparameter options
param_grid = dict(max_iter=max_iter,C=C)

# Grid search
random = RandomizedSearchCV(estimator=lrg, param_distributions=param_grid, cv = 10, scoring = 'accuracy')


In [ ]:
random_result = random.fit(X_train, y_train)
# Summarize results
print("Best: %f using %s" % (random_result.best_score_, random_result.best_params_))

In [ ]:
# Evaluate the classifier by using accuracy measure
# Apply the trained Classifier to the test data (which, remember, it has never seen before to measure accuracy)
print("classifier accuracy:", random.score(X_test, y_test))  # score model

In [ ]:
from sklearn.metrics import confusion_matrix

# predict with best model
# Evaluate the classifier by using confusion matrix compare y predicted to actual y values

# Apply the trained Classifier to the X-test data and make predictions
y_pred = random.predict(X_test)  # use model to predict on test data for generalisation
y_true = y_test
# create confusion matrix

confusion_matrix(y_true, y_pred)

In [ ]:
pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Predicted'], margins=True)

In [ ]:
# precision, recall, F1-score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report

precision_recall_fscore_support(y_true, y_pred)

In [ ]:
target_names = ['class 0', 'class 1']
print(classification_report(y_true, y_pred, target_names=target_names))

In [ ]:
# RandomForest classifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
import xgboost as xgb
from sklearn.metrics import roc_auc_score
# Create a random forest Classifier. By convention, rf, put hyperparameters, default
rf = RandomForestClassifier(random_state=0)

# Train the Classifier to take the X-training features and learn how they 
# relate to the y-training target independent variable

rf.fit(X_train, y_train)  # fit model

In [ ]:
# Evaluate Randomforest
print("classifier accuracy:", rf.score(X_test, y_test))  # score model

In [ ]:
# Apply the trained Classifier to the X-test data and make predictions
rfy_pred = rf.predict(X_test)  # use model to predict on test data for generalisation
y_true = y_test
# create confusion matrix

confusion_matrix(y_true, rfy_pred)

In [ ]:
print(classification_report(y_true, rfy_pred, target_names=target_names))

In [ ]:
# plot of precission and Recall curve 

from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt

precision, recall, _ = precision_recall_curve(y_test, rfy_pred)

plt.step(recall, precision, color='b', alpha=0.2,
         where='post')
plt.fill_between(recall, precision, step='post', alpha=0.2,
                 color='b')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title('2-class Precision-Recall curve')

In [ ]:
# ROC curve
fpr_rf, tpr_rf, _ = roc_curve(y_test, rfy_pred)
roc_auc_rf = auc(fpr_rf, tpr_rf)
plt.figure(figsize=(8,8))
plt.xlim([-0.01, 1.00])
plt.ylim([-0.01, 1.01])
plt.plot(fpr_rf, tpr_rf, lw=1, label='{} curve (AUC = {:0.2f})'.format('RF',roc_auc_rf))


plt.xlabel('False Positive Rate', fontsize=16)
plt.ylabel('True Positive Rate', fontsize=16)
plt.title('ROC curve', fontsize=16)
plt.legend(loc='lower right', fontsize=13)
plt.plot([0, 1], [0, 1], color='navy', lw=1, linestyle='--')
plt.axes().set_aspect('equal')
plt.show()

In [ ]:
# feature Importance
# Rank of the importance of feature
rf.feature_importances_

In [ ]:
# feature importance

feature_importances = pd.DataFrame(rf.feature_importances_,index = features_df.columns,
                                   columns=['importance']).sort_values('importance',ascending=False)

feature_importances.head(34)

In [ ]:
#Import knearest neighbors Classifier model
from sklearn.neighbors import KNeighborsClassifier

#Create KNN Classifier
knn = KNeighborsClassifier(n_neighbors=7)

#Train the model using the training sets
knn.fit(X_train, y_train)

#Predict the response for test dataset
knny_pred = knn.predict(X_test)

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, knny_pred))

In [ ]:
print(classification_report(y_true, knny_pred, target_names=target_names))

In [ ]:
# The more data intensive estimators gave better performance precision and recall than logistic regresion 
# Comparison of prediction accuracy by the models shows that logistic regression, random Forest and K nearest Neighbour
# gave the similar performance accuracy
# based on the data.

# Accuracy on obtained are as follws logistic regression 0.9361228698578096, RandomForest 0.9810593726256377 
# and K Nearest neighbour 0.968305655052643
# The results from these models shows that with more data, feature engineering and hyperparameter tunning
# on RandomForest and KNN, the performance will be improved.

In [ ]:
# plot hyper parameter test random Forest
'''
N_estimators
n_estimators represents the number of trees in the forest.
Usually the higher the number of trees the better to learn the data.
However, adding a lot of trees can slow down the training process considerably,
therefore we do a parameter search to find the sweet spot.
'''
n_estimators = [1, 2, 4, 8, 16, 32, 64, 100, 500]
train_results = []
test_results = []
for estimator in n_estimators:
   rf = RandomForestClassifier(n_estimators=estimator, n_jobs=-1)
   rf.fit(X_train, y_train)
   train_pred = rf.predict(X_train)
   false_positive_rate, true_positive_rate, thresholds = roc_curve(y_train, train_pred)
   roc_auc = auc(false_positive_rate, true_positive_rate)
   train_results.append(roc_auc)
   y_pred = rf.predict(X_test)
   false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
   roc_auc = auc(false_positive_rate, true_positive_rate)
   test_results.append(roc_auc)
from matplotlib.legend_handler import HandlerLine2D
line1, = plt.plot(n_estimators, train_results, label= "Train AUC")
line2, = plt.plot(n_estimators, test_results, label="Test AUC")
plt.legend(handler_map={line1: HandlerLine2D(numpoints=2)})
plt.ylabel("AUC score")
plt.xlabel("n_estimators")
plt.show()


In [ ]:
#max_depth
'''
max_depth represents the depth of each tree in the forest. The deeper the tree,
the more splits it has and it captures more information about the data. 
We fit each decision tree with depths ranging from 1 to 32 and plot 
the training and test errors.
'''
max_depths = np.linspace(1, 100, 100, endpoint=True)
train_results = []
test_results = []
for max_depth in max_depths:
    rf = RandomForestClassifier(n_estimators= 100,max_depth=max_depth, n_jobs=-1)
    rf.fit(X_train, y_train)
    train_pred = rf.predict(X_train)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(y_train, train_pred)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    train_results.append(roc_auc)
    y_pred = rf.predict(X_test)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    test_results.append(roc_auc)
from matplotlib.legend_handler import HandlerLine2D
line1, = plt.plot(max_depths, train_results, "b", label="Train AUC")
line2, = plt.plot(max_depths, test_results, "r", label="Test AUC")
plt.legend(handler_map={line1: HandlerLine2D(numpoints=2)})
plt.ylabel("AUC score")
plt.xlabel("Tree depth")
plt.show()
